In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.metrics.pairwise import cosine_similarity
from typing import *
import copy
from deeppavlov.dataset_readers.ontonotes_reader import OntonotesReader

/home/kostrovskiy/prog/AI/NLP/DeepPavlov/env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
[nltk_data] Downloading package punkt to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
2018-09-07 17:17:33.657 DEBU

In [2]:
from src.fewshot_ner_viz_component.data_processing import *
from src.fewshot_ner_viz_component.utils import *
from src.fewshot_ner_viz_component.fewshot_ner_binary_classifier import *

In [3]:
dataset_orig = read_data()
ne_type = 'PERSON'
dataset = filter_dataset_by_ne_types(dataset_orig, ne_type)

Num of train sentences: 75187
Num of valid sentences: 9603
Num of test sentences: 9479
[(['Actions', 'had', 'to', 'be', 'taken', 'to', 'break', 'through', 'the', 'blockade', '.'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']), (['On', 'a', 'night', 'in', 'late', 'July', '1940', ',', 'the', 'atmosphere', 'in', 'Zhuanbi', 'Village', 'in', 'Shaanxi', 'was', 'unusual', '.'], ['O', 'B-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'O', 'O', 'O', 'O', 'B-GPE', 'I-GPE', 'O', 'B-GPE', 'O', 'O', 'O']), (['Villager', 'Xiao', 'Jianghe', 'has', 'a', 'vivid', 'memory', 'of', 'this', 'piece', 'of', 'history', '.'], ['O', 'B-PERSON', 'I-PERSON', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']), (['On', 'that', 'dark', 'night', ',', 'everyone', 'was', 'sleeping', 'when', 'human', 'voices', 'and', 'neighing', 'horses', 'were', 'heard', 'within', 'the', 'village', '.'], ['O', 'B-TIME', 'I-TIME', 'I-TIME', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

In [4]:
N_EXAMPLES = [10]
N_TEST_SENTENCES = 100
N_SAMPLES = 1
MODELS = ['ne_centroid', 'weighted_kNN', 'centroid_kNN', 'svm']
# MODELS = ['weighted_kNN', 'svm']
MODELS = ['svm']
MODEL_PARAMS = {'ne_centroid': {'sim_type': 'cosine'},
               'ne_nearest': {'sim_type': 'cosine'},
               'weighted_kNN': {'k': 1, 'metric': 'cosine', 'use_class_weights': False, 'use_sim_weights': True},
               'centroid_kNN': {'k': 10, 'metric': 'dot_prod', 'use_class_weights': False},
               'svm': {}}

In [5]:
def flatten_with_mask(seq_mat: np.ndarray, mask):
    return seq_mat[mask == 1]

In [6]:
def eval_dataset(ner_clf, batch_generator, data_type='test', batch_size=32, models=['svm'], model_params={}):
    """
        Calculate F1 scores for the whole dataset
    """
    print('Evaluation on {} dataset'.format(data_type))
    if not isinstance(models, list):
        models = [models]
    pred_total = {model: [] for model in models}
    true_total = []
    n_batches = ceil(len(dataset[data_type])/batch_size)
    for i, (tokens, tags, mask, y) in enumerate(batch_generator.gen_batches(batch_size, data_type=data_type, shuffle=False, binaryTags=True)):
        results = ner_bin_clf.predict(tokens, models, model_params)
        for model in models:
            pred = results[model]['pred']
            pred_total[model] = np.concatenate([pred_total[model], pred])
            print(pred.shape)
        print(flatten_with_mask(y, mask).shape)
        true_total = np.concatenate([true_total, flatten_with_mask(y, mask)])
        sys.stdout.write('\r')
        progress = i/n_batches
        sys.stdout.write("[%-20s] %d%%" % ('='*int(ceil(progress*20)), ceil(progress*100)))
        sys.stdout.flush()
    f1_total = {model: f1_score(true_total, pred_total[model]) for model in models}

    return {'f1': f1_total}

### Make samples

In [7]:
data_samples = []
for i, n_examples in enumerate(N_EXAMPLES):
    data_samples.append([])
    for j in range(N_SAMPLES):    
        tokens_train,tags_train = get_data_sample(dataset['train'], n_examples)
        tokens_test,tags_test = get_data_sample(dataset['test'], min(N_TEST_SENTENCES, len(dataset['test'])))   # TODO: Evalute on whole test dataset
        y_test = np.array([1 if t == 'T' else 0 for seq in tags_test for t in seq])
        data_samples[-1].append({'train': {'tokens': tokens_train, 'tags': tags_train}, 'test': {'tokens': tokens_test, 'tags': tags_test, 'y': y_test}})

### Prepare embedder params

In [8]:
# ELMo vs GLoVe vs fastText
embedder_params=[{'params': {'use_elmo': True, 'use_glove': False, 'use_fastText': False, 'use_cap_feat':False}, 'name': 'elmo'},
                 {'params': {'use_elmo': False, 'use_glove': True, 'use_fastText': False, 'use_cap_feat':False}, 'name': 'glove'},
                 {'params': {'use_elmo': False, 'use_glove': False, 'use_fastText': True, 'use_cap_feat':False}, 'name': 'fastText'}]
embedder_params=[{'params': {'use_elmo': True, 'use_glove': False, 'use_fastText': False, 'use_cap_feat':False}, 'name': 'elmo'},
                 {'params': {'use_elmo': False, 'use_glove': True, 'use_fastText': False, 'use_cap_feat':False}, 'name': 'glove'}]

In [9]:
# ELMo, ELMo+GloVe, ELMo+Cap, ELMo+GloVe+Cap
# embedder_params=[{'params': {'use_elmo': True, 'use_glove': False, 'use_cap_feat':False}, 'name': 'ELMo'},
#                  {'params': {'use_elmo': True, 'use_glove': True,  'use_cap_feat':False}, 'name': 'ELMo+GloVe'},
#                  {'params': {'use_elmo': True, 'use_glove': False, 'use_cap_feat':True}, 'name':  'ELMo+Cap'},
#                  {'params': {'use_elmo': True, 'use_glove': True,  'use_cap_feat':True}, 'name':  'ELMo+GloVe+Cap'}]

In [10]:
dataset_iterator = DatasetIterator(dataset)

In [11]:
f1_scores = {}
for i, embedder_params_cur in enumerate(embedder_params):
    print('============ {} ============'.format(embedder_params_cur['name']))
    params = embedder_params_cur['params']
    label = embedder_params_cur['name']
    tf.reset_default_graph()
    embedder = CompositeEmbedder(**params)
    f1_scores[label] = []
    for j, n_examples in enumerate(N_EXAMPLES):
        print('+++ Number of examples: {} +++'.format(n_examples))
        f1_scores[label].append({})
        for model in MODELS:
            f1_scores[label][j][model] = {'vals': [], 'avg': 0, 'std': 0}
        for k, data_sample in enumerate(data_samples[j]):
            print('Sample {}/{}'.format(k+1, N_SAMPLES))
            tokens_train = data_sample['train']['tokens']
            tags_train = data_sample['train']['tags']
#             tokens_test = data_sample['test']['tokens']
#             tags_test = data_sample['test']['tags']
#             y_test = data_sample['test']['y']
            ner_bin_clf = FewshotNerBinaryClassifier(embedder, svm_kernel=cosine_similarity, verbose=False)
            ner_bin_clf.train_on_batch(tokens_train, tags_train)
#             results = ner_bin_clf.predict(tokens_test, MODELS, MODEL_PARAMS)
            eval_results = eval_dataset(ner_bin_clf, dataset_iterator, data_type='test', batch_size=64, models=MODELS, model_params=MODEL_PARAMS)
            for model in MODELS:    
                f1 = eval_results['f1']['model']
                f1_scores[label][j][model]['vals'].append(f1)
                print('Model: {}, F1 (tokens) = {:.2f}'.format(model, f1*100))
            del ner_bin_clf
        print('Average F1 scores:')
        for model in MODELS:
            f1_avg = np.mean(f1_scores[label][j][model]['vals'])
            f1_std = np.std(f1_scores[label][j][model]['vals'])
            f1_scores[label][j][model]['avg'] = f1_avg
            f1_scores[label][j][model]['std'] = f1_std
            print('Model: {}, avg F1 (tokens) = {:.2f}, std = {:.2f}'.format(model, f1_avg*100, f1_std*100))
    del embedder

============ elmo ============
INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.


2018-09-07 17:17:48.180 INFO in 'tensorflow'['tf_logging'] at line 160: Using /tmp/tfhub_modules to cache modules.


INFO:tensorflow:Initialize variable module/aggregation/scaling:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with aggregation/scaling


2018-09-07 17:17:48.606 INFO in 'tensorflow'['tf_logging'] at line 116: Initialize variable module/aggregation/scaling:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with aggregation/scaling


INFO:tensorflow:Initialize variable module/aggregation/weights:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with aggregation/weights


...

2018-09-07 17:17:48.983 INFO in 'tensorflow'['tf_logging'] at line 116: Initialize variable module/bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/projection/kernel:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/projection/kernel


INFO:tensorflow:Initialize variable module/bilm/char_embed:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/char_embed


2018-09-07 17:17:48.999 INFO in 'tensorflow'['tf_logging'] at line 116: Initialize variable module/bilm/char_embed:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/char_embed


+++ Number of examples: 10 +++
Sample 1/1
Evaluation on test dataset
(4160,)
(1532,)
[                    ] 0%(3904,)
(1298,)
[=                   ] 4%(5376,)
(1364,)
[==                  ] 8%(5184,)
(1399,)
[===                 ] 12%(5312,)
(1309,)
[====                ] 16%(3328,)
(1187,)
[====                ] 20%(4864,)
(1444,)
[=====               ] 24%(5056,)
(1585,)
[======              ] 29%(5120,)
(1524,)
[=======             ] 32%(2816,)
(1329,)
[========            ] 36%(3264,)
(1765,)
[========            ] 40%(4864,)
(2175,)
[=========           ] 44%(3328,)
(1946,)
[==========          ] 48%(4160,)
(1982,)
[===========         ] 52%(3456,)
(1961,)
[============        ] 57%(3968,)
(1914,)
[============        ] 60%(3712,)
(1547,)
[=============       ] 64%(5504,)
(2165,)
[==============      ] 68%(4480,)
(1485,)
[===============     ] 72%(2816,)
(873,)
[================    ] 76%(3968,)
(1394,)
[================    ] 80%(5696,)
(1234,)
[=================   ] 84%(4608,)
(14

KeyboardInterrupt: 

In [ ]:
print(f1_scores)

In [ ]:
def make_table_caption(labels):
    s = '{:<8s}'
    for lab in labels:
        s += '\t{:<8s}'
    return s.format('MODEL', *labels)

def make_table_line(model, scores):
    s = '{:<8s}'
    for score in scores:
        s += '\t{:<8.2f}'
    return s.format(model, *scores)

In [ ]:
labels = [par['name'] for par in embedder_params]
print(make_table_caption(labels))
for i, n_examples in enumerate(N_EXAMPLES):
    print('+++ Number of examples: {} +++'.format(n_examples))
    for model in MODELS:
        print(make_table_line(model, [f1_scores[lab][i][model]['avg']*100 for lab in labels]))

In [ ]:
# create plot
fig, ax = plt.subplots()
index = np.arange(len(MODELS))*2
bar_width = 0.35
opacity = 0.8

for i, label in enumerate(labels):
    f1 = [f1_scores[label][-1][model]['avg']*100 for model in MODELS]
    rects1 = plt.bar(index + i*bar_width, f1, bar_width,
                     alpha=opacity,
                     label=label)
 
plt.xlabel('Models')
plt.ylabel('F1 scores (token-wise)')
plt.title('F1 scores for different feature vectors')
plt.xticks(index + bar_width, MODELS)
plt.legend()
 
plt.tight_layout()
plt.show()